In [10]:
%pwd

'/home/aistudio'

In [1]:
# 测试paddle conv2d 参数;  
# 完成 整个 conv+pool block;  finish 测试
# 本周开始搭建视觉模型；paper list;
# 整体训练框架;
# alex next 
# vgg 
# googleNet 
# resnet 
# inception 
# dpn 
# SE-ResNeXt 
# https://www.cnblogs.com/vincent1997/p/10901875.html
# https://cloud.tencent.com/developer/article/1382720
# https://blog.csdn.net/hua111hua/article/details/89422661#PaddlePaddleVisualDL_132 可视化

In [2]:
import sys
sys.path.append('/home/aistudio/work')
from models.model import conv_pool_layer, alex_net
from models.utils import load_image, to_rgb_img
from models.data import image_reader_creator

import paddle.fluid as fluid
import paddle
from paddle.fluid.initializer import TruncatedNormalInitializer
import numpy as np
import sys
import os
from glob import glob
from multiprocessing import cpu_count
import matplotlib.pyplot as pyplot
import subprocess

In [3]:
gt_dir = '/home/aistudio/work/task3/train/truth_pic/'
fk_dir = '/home/aistudio/work/task3/train/rumor_pic/'
revert_gt_dir = '/home/aistudio/work/task3/train/truth_revert/'
revert_fk_dir = '/home/aistudio/work/task3/train/fake_revert/'
truth_samples = [(i, 1) for i in glob(gt_dir+'*')[10:20]]
fake_samples = [(i, 0) for i in glob(fk_dir+'*')]
truth_revert_samples = [(i, 1) for i in glob(revert_gt_dir+'*')]
fake_revert_samples = [(i, 0) for i in glob(revert_fk_dir+'*')]
all_samples = [sample for sub_lis in [truth_samples, fake_samples, truth_revert_samples, fake_revert_samples] for sample in sub_lis]
num_cls = 2
bs = 100


In [4]:
main_program = fluid.Program()
startup_program = fluid.Program()
with fluid.program_guard(main_program=main_program, startup_program=startup_program):
    data = fluid.layers.data(name='data', shape=[3,100,100], dtype='float32')
    label = fluid.layers.data(name="label", shape=[1], dtype='int64')
    net_out = alex_net(data)
    pred = fluid.layers.fc(input=net_out, size=num_cls, act='softmax',
                            param_attr=paddle.fluid.ParamAttr(name="param1", initializer=TruncatedNormalInitializer()))
    
    loss = fluid.layers.cross_entropy(input=pred, label=label)
    ave_loss = fluid.layers.mean(loss)
    
    batch_size = fluid.layers.create_tensor(dtype='int64')
    batch_acc = fluid.layers.accuracy(input=pred, label=label, total=batch_size)
    
    sgd = fluid.optimizer.AdamOptimizer(learning_rate=0.001)
    sgd.minimize(ave_loss)
        

chnnels:3
chnnels:64
chnnels:192
chnnels:384
conv4 shape:(-1, 256, 3, 3)


In [5]:
from visualdl import LogWriter
logdir = "/home/aistudio/work/logs/"
logwriter = LogWriter(logdir, sync_cycle=10)

# 创建loss的趋势图
with logwriter.mode("train") as writer:
    loss_scalar = writer.scalar("loss")

# 创建acc的趋势图
with logwriter.mode("train") as writer:
    acc_scalar = writer.scalar("acc")

# 创建可视化的训练模型结构
with logwriter.mode("train") as writer:
    param1_histgram = writer.histogram("param1", 100)

In [8]:
use_cuda = True
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
exe = fluid.Executor(place)

reader = image_reader_creator(all_samples, len(all_samples))
batch_reader = paddle.batch(reader, 
bs)
feeder = fluid.DataFeeder(place=place, feed_list=[data, label])

train_step = 0
sample_num = 0
param1_var = startup_program.global_block().var("param1")

exe.run(startup_program)
accuracy = fluid.average.WeightedAverage()
for epoch in range(4):
    accuracy.reset()
    for bs_data in batch_reader():
        
        bs_loss, param1, bs_acc, bs_weight = exe.run(main_program,
                                                       feed=feeder.feed(bs_data),
                                                       fetch_list=[ave_loss, param1_var, batch_acc, batch_size])
        print('loss:{}\n acc:{}\n'.format(bs_loss, bs_acc))                                               
        loss_scalar.add_record(train_step, bs_loss[0])
        acc_scalar.add_record(train_step, bs_acc[0])
        param1_histgram.add_record(train_step, param1.flatten())
        train_step+=1

        accuracy.add(value=bs_acc, weight=bs_weight)
        pass_acc = accuracy.eval()


/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/paddle/fluid/average.py:65: Warning: The WeightedAverage is deprecated, please use fluid.metrics.Accuracy instead.
  (self.__class__.__name__), Warning)


loss:[3.5012004]
 acc:[0.58]

loss:[0.02648523]
 acc:[0.98999995]

loss:[0.12764464]
 acc:[0.98999995]

loss:[0.]
 acc:[1.]

loss:[1.1237506e-06]
 acc:[1.]

loss:[0.00235029]
 acc:[1.]

loss:[0.00295741]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.26815924]
 acc:[0.98999995]

loss:[0.64]
 acc:[0.98999995]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[8.94063e-05]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[2.2790894e-06]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss:[0.]
 acc:[1.]

loss

In [7]:
!visualDL --logdir=/home/aistudio/work/logs/ --port=8080

/bin/sh: 1: visualDL: not found
